# Why Haskell? 

> The purpose of abstraction is not to be vague, but to create a new semantic level in which one can be absolutely precise.
> - Edsger Dijkstra

The short answer is that I believe that:

## Haskell is the Best Available Language for Writing "Boring Line of Business" Applications

This might be a suprising claim for people that have looked into Haskell previously.  Based on it's reputation as a weird academic language you might expect to hear the opposite -- that it excels in certain esoteric or specialized (maybe "mathy") domains but not so much for "practical" situations like building a CRUD web app.

First, I will say that Haskell isn't all sunshine and roses.  I think the biggest problem is that Haskell seems to strange and scary to experienced programmers coming from other languages.  If you are already familiar with Java and you decide to learn python not only do a lot of the concept translate pretty directly but the areas where they differ allow you to easily compare them and evaluate the tradeoffs.

Encountering Haskell is more like the type of situation that arises in sci-fi movies where the heroes encounter aliens that have strange (usually green) alien-technology versions of all of the familiar weapons and tools they are used to.  Inevitabiy the alien weapons have more fire power than the human versions -- eg. they are energy weapons instead of bullets driven by gun powder, they are indestructible, they have limitless power sources, and so on.  

<img width="500" height="262" src="http://www.makecnc.com/prodimages/megatronic_ray_gun_plans_1000.jpg">

Maybe the first time a human tries to use one of the alien guns they get shocked or they blow a huge hole in the wall or have some other experience that frightens them away from trying to use the alien tech for their own purposes.  In the movies though there's usually some otherwise hopeless situation that forces the humans to resort to harnessing the superior alien technology to save the themselves or more likely the entire planet.

It's relatively rare to encounter such dire circumstances in the world of programming languages, so the unfortunate reality is that most developers coming from other languages tend to be gunshy and reluctant to press on after their initial encounter with Haskell.  The good news is that unlike literal alien technology, while the people that have created Haskell and related technology might *seem* like aliens to us, they're really just super nerdy humans -- which means a) if they can understand it so can we, and b) they're actually around, available to help us and usually friendly to our cause.

This inability for experienced developers from other languages to easily get up to speed leads to a much smaller community than other mainstream languages, and the smaller community leads to a whole host of knock-on effects around eg. tooling, learning resources, opportunities for learning on the job, inability to ramp up large teams quickly, etc.

The rest of this notebook will be my attempt to justify and clarify my headline claim above and maybe convince you that like crazy high-tech alien energy weapons, Haskell affords us such a high degree of increas leverage that these disadvantages are easily outweighed by the advantages in the long run (even if we're "just" building CRUD web apps).

Hopefully if you're reading (or hearing) this you've already been through my Taste of Haskell introductory material and you came away with the understanding that quite a lot can be accomplished in Haskell using tools that are similar to those that you are familiar with from other languages.  In the sections that follow I hope to elaborate on some of the differences and what benefits those differences bring to the experience of developing software in order to justify the effort required to understand their strangeness.

Ultimately I think the only sure way to really grok what I'm trying to communicate is to dive in and get over the hurdle of the strangeness and get to the point where you start to experience the benefits first hand -- so I will consider it to be a success if I convince you that such a journey might be worth embarking on.

## Let's get started.

Any good Haskell module/blog post/notebook begins with a raft of language pragmas (which enable various extensions to the core language) and import statements.  As a quick aside, these language pragmas are often one of the things that seem to put people off the language a bit.  I see a lot of posts even from people that like Haskell complaining that dealing with the pragmas/extensions is one of the worst parts of Haskell.  I really don't understand what the big deal here is.  In practice you don't have to worry about language extensions until you want to do something that requires them, then while learning that thing you will learn about the extension you need and you'll put in your project (either at the top of the file or in your .cabal file) and you're good to go.  There are a few language extensions that are contentious for various reasons but for the most part no one uses them and you don't have worry about them.

One of the warts of Haskell is that the default Prelude (think "standard library") includes a number of partial functions.  This basically means pure functions that can fail to return an answer of expected type for one reason or another (usually throw an exception or loop forever).  It's possible to write Haskell using the default Prelude and just avoid these through conscientiousness but it's a lot easier to just switch out the Prelude for one that replaces these functions with total versions.  Protolude is one such alternative Prelude and it doesn't make any controversial choices so it's a good choice for those starting out or for basing your own custom Prelude on.

We'll add additional pragmas and imports throughout the document so there's at least some opportunity to understand what they are doing.

In [1]:
:ext NoImplicitPrelude

import           Protolude
import qualified Prelude   as P  -- Expose the original Prelude so we can use naughty functions when convenient

## Concepts We're Going to Cover

- Succinctness / Terseness
- Static Typing (with Type Inference)
  - No "billion dollar mistake" (null)
- Algebraic Data Types
- Purity / Referential Transparency
- Parametricity
- Totality
- Explicit Effects
- Laziness
- Immutability
- Pattern Matching

After we get through these we'll dig into some examples that highlight the advantages these features provide in practice:

- The StreamSource Algebraic Data Type 
- Haxl
- Fun with Optics (Lenses)
- Dependently Typed Tensorflow
- Property-based Testing with QuickCheck
- Idris (Beyond Haskell)

## Succinctness / Terseness

- BENEFIT: Easier to comprehend code when there's less of it.
- BENEFIT: Easier to compose small bits of functionality into larger solutions when there's less noise in the way.

One of the defining charateristics of (well written) Haskell code is that it is succinct or terse.  People tend to use the word terse but I find that that terseness often has a bit of negative connotation attached -- as it the code is "too terse" or "terse code is hard to read".  I think "succinctness" is a better term and conveys more clearly that our ultimate goal is clarity.  It's easy to imagine (and write!) code that is too terse but hard to imagine code that is "too succinct" or "too clear".

This is an example from Aaron Contorer the CEO of FP Complete via one of their marketing videos.  It's obviously fairly contrived but I would argue that it is a fairly accurate representation of the types of differences you see when rewriting code "in the small" from other statically typed languages (Javas in this case) to Haskell:

```java
final int LIMIT = 50;
int [] a = new int[LIMIT];
int [] b = new int[LIMIT - 5];

for (int i=0; i<LIMIT; i++) {
  a[i] = (i+1) * 2;
  if (i >= 5) {
    b[i-5] = a[i];
  }
}
```

vs

```haskell
let a = [2,4..100]
let b = drop 5 a
```

It's clear at a glance what the Haskel code is doing whereas even an experienced Java developer will take a few moments to step through the Java code trying to reason about what the loops are doing and figuring out what the final output would be.  

The difference is even more pronounced instead of just reading the code casually you are reading it in a situation like a code review or because you are trying to track down a bug.  In the Java code you have to start checking whether the bounds of the array are declared properly, whether each part of the loop declaration is correct, and so forth and so on.  In the Haskell code you got most of that in the same first glance.  There's just far less room for bugs to hide.

It's worth noting here that in modern Java you can probably write something much closer to the Haskell code, perhaps something like this:

```java
import java.util.stream.IntStream;
import java.util.Arrays;

final int[] a = IntStream.range(2, 100).filter(n -> n % 2 == 0).toArray();
final int[] b = Arrays.stream(a).filter(n -> n > 10).toArray();
System.out.println(Arrays.toString(a));
System.out.println(Arrays.toString(b));
```

It may be possible write this even more succinctly in Java but this is about the best I could do having been free of Java for more than a decade.  

As you can see not only is it still considerably more cumbersome than the equivalent Haskell code but it's not even really equivalent:

- In the Haskell code the initial list from 2-100 is generated once, and then "drop 5" version just uses the fifth element as the head of a new list.  The 'b' list is entirely shared with the same range from the 'a' list.
- There's also not this conversion to/from Arrays.  The conversion toArray at the end is not only necessary to print it out but also because IntStreams can only be consumed once in Java.  Then we have to convert back to a stream to do the filtering to produce the 'b' list.  Then we have to convert back to an array to print them out... using an awkward `Arrays.toString` helper which adds another wart.  
- On top of that there's no built in "drop" method for streams so I had to resort to the "filter" hack which doesn't scale as I had to stop and do the math to figure out which the fifth value would be -- which works in this trivial example but wouldn't if the calculation were any more complex.  

Almost all of the extraneous noise here is in service of communicating to the Java compiler how to do what you want to do, whereas the Haskell version reads like a description of the result you want with little to no information about how it should be accomplished.  Hopefully this benefit is obvious even in small samples of code like this but the ability to focus on expressing the business goal rather than constructing a machine to accomplish the business goal becomes really valuable as you scale to larger and larger amounts of code.

The last bullet point above could be worked around by implementing a "drop" method or function in your own library but even this appears to be much more awkward than it would be in Haskell, where the naive recursive solution would be:

In [2]:
recDrop _ [] = []
recDrop n (x:xs)
  | n <= 0    = x:xs
  | otherwise = recDrop (n - 1) xs
  
print (recDrop 5 [1..10])  

[6,7,8,9,10]

And there are a million other ways to achieve the same thing easily with other built in functions, eg something like this which takes advantage of `dropWhile` (which is a method on java's streams) but does so by zipping the list with it's index and then unzipping it (which would, as far as I can tell, be very cumbersome in java (since again, there's no existing `zip`, `fst`, `snd` etc):

In [3]:
zipDrop n = map snd . dropWhile ((< n) . fst) . zip [0..]

print (zipDrop 5 [1..10])

[6,7,8,9,10]

As a further aside, even if we didn't have the range notation that allows us to express even numbers easily, I would argue the Haskell version of `IntStream.range(2, 100).filter(n -> n % 2 == 0)` is much more expressive and easier to understand at a glance than the java version:

In [4]:
filter even [1..100]

[2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100]

Hopefully it's obvious what the original example does but just for completeness:

In [5]:
let a = [2,4..100]
let b = drop 5 a

print a
print b

[2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100]

[12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100]

Obviously you get similar advantages when rewriting Java in many dynamic languages, like say Ruby or Python.  The biggest difference is that the Haskell code still has all of the advantages of static typing that the Java code had -- and more.  Which leads us to...

## Static Typing (with Type Inference)

- BENEFIT: Catch mistakes/issues earlier in the process, closer to where you made them.
- BENEFIT: (of Type Inference) Less typing / noise as compared to popular forms of Static Typing (eg Java, C#, etc).

Most language features represent a set of tradeoffs and Static Typing in general is no different.  I think most people think of languages like Java or C# when they think of statically typed languages.  However the experience of static typing as implemented in Haskell is quite different from the experience of static typing in those languages.

One of the major reasons for the difference is type inference.  Haskell can infer the types of most values (including functions) in most situations.  This is why were able to make it through almost the entire introductory portion without ever having to write a type signature.  If it were possible to add type inference to Java in a principled way I think it would go a long way towards making the experience of writing Java code as nice as the experience of writing Python code.

Haskell's type system also has a number of advanced features which allow us to leverage the type system to even greater effect.  For example, per [this blog post](https://medium.com/@johnmcclean/simulating-higher-kinded-types-in-java-b52a18b72c74) this is what we would like to (but cannot) write as a generalized definition for `Functor` in Java:

```java
public interface Functor<C> {
     public <T,R> C<R> map(Function<? super T, ? extends R> mapper, C<T> genericType);
}
```

Not only is it impossible to write but even if it were possible the noise of the rest of Java's syntax makes it hard for even a competent Java program to see what's going on without quite a bit of scrutiny and thought, whereas the actual definition of the `Functor` typeclass in Haskell is this:

In [6]:
class MyFunctor f where
  myfmap :: (a -> b) -> f a -> f b

...which more or less says what it does: "`f` is a (My)`Functor` if you can implement a function called (my)`fmap` which takes a function from `a` to `b` and an instance of (My)`Functor` of `a`'s and returns a (My)`Functor`  of `b`'s.

Functor is one of (if not the) most fundamental class in Haskell and it's used everywhere.  Some common examples are:

In [7]:
print $ fmap (*3) [1..10]
print $ fmap (*3) (Just 5)
print $ fmap (*3) Nothing
print $ fmap (*3) (Right 7)
print $ fmap (*3) (Left "hello")

[3,6,9,12,15,18,21,24,27,30]

Just 15

Nothing

Right 21

Left "hello"

A particularly interesting example is IO actions, which we haven't gotten to yet, but to tease a little, here is an IO action (which doesn't actually do any IO) that takes a number and adds one to it:

In [8]:
askPlusOneIO :: IO Int
askPlusOneIO = do
  n <- P.read <$> P.getLine
  return (n + 1)
  
askPlusOneIO >>= print

11

and we can `fmap` over it to get an IO action that does the same thing then multplies by three:

In [9]:
askPlusOneTimesThreeIO :: IO Int
askPlusOneTimesThreeIO = fmap (*3) askPlusOneIO

askPlusOneTimesThreeIO

33

...which leads us to:

## Algebraic Data Types

- BENEFIT: Makes it easier to model your domain accurately.
- BENEFIT: Makes it easier to create non-leaky abstractions.

There are no "special types" in Haskell -- there is syntax sugar for lists, tuples and numbers but they could all be implemented directly as ADTs if they didn't already exist (just with more awkward syntax/apperance).

The primary benefit of ADTs is that once you are familiar with them they make domain modeling easier than in most other languages.

A simple example of this is modeling the situation of loading a list of results from the network.  In JavaScript you might be tempted to do something like this:

In your business logic:

```javascript
var curState = { items: [] };  // empty list = not yet fetched
      
fetchItems(curState);
```

In your view (probably in template logic, but you get the idea):

```javascript
if (curState.items.length == 0) {
  renderLoadingScreen();
} else {
  renderTheItems(curState.items);
}
```

The problem here is that if the actual data fetch returns an empty list then you will continue to render the loading screen instead of an empty list, or perhaps better a "No Items Exist" null state view that provides helpful hints on creating your first item.

This isn't that hard to fix in JS... you could add as separate flag for whether the items have been fetched and be on your way.  The problem is that as you run into more and more little issues like this (eg. how do you handle the case where there was an error fetching the data?) your code will accrete more and more special case flags, and it won't always be clear what an empty list means, or when you need to add logic to look at which flags, etc.

With Algebraic data types we can model the situation like this:

In [10]:
type Item = () -- dummy value

data CurState
  = Loading
  | LoadingError Text
  | ItemsLoaded [Item]

Now, everywhere that you deal with `CurState` you will be forced to use pattern matching to deconstruct the value and handle each case.  If you miss a case the compiler will let you know:

In [11]:
state = Loading

case state of
  Loading           -> "Loading..."
  ItemsLoaded items -> "Loaded " ++ show (length items) ++ " items."

"Loading..."

Unfortunately IHaskell disables warnings because the nature of a notebook causes a plethora of them, and if you enable them it displays them in the console instead of the notebook, but if we had them enabled we would see something like this:

    <interactive>:1:1: warning: [-Wincomplete-patterns]
        Pattern match(es) are non-exhaustive
        In a case alternative: Patterns not matched: (LoadingError _)

ADTs have some ancillary benefits like enabling you to think through the space of your types algebraically all ADTs are a "sum of products" -- each additional constructor adds to the number of possible values and each additional parameter in a given constructor multiplies the number of values that constructor can produce.  For example:

In [12]:
data Foo
  = Bar Char Bool
  | Baz Char
  deriving (Eq, Ord, Read, Show)  

The type `Foo` can be constructed with the `Bar` constructor which takes a `Char` and a `Bool`.  A char can be any unicode value but for the moment lets pretend they are restricted to the 256 ASCII characters.  In this case the `Bar` constructor can take on 256 chars _times_ one of 2 bools, for a total of 512 possible values.  The `Baz` constructor can take on only the 256 possible char values.  Together the total space of `Foo` is the 512 possible values for `Bar` _plus_ the 256 possible values of `Baz` for a total of 768 possible values.

Other languages that have them all have other problems: Idris/Agda not production ready yet, no ecosystem, libraries.  Scala/FSharp mixes in OOP too -- on the Scala side this makes things way more complex, on the fsharp side this makes things less powerful.

## Purity

- BENEFIT: Referential transparency
- BENEFIT: Enables equational reasoning
- BENEFIT: Enables honest parametricity

A pure function is one that behaves like a mathematical function -- that is, any time you give it the same arguments you will get the same result back.  This is as opposed to "functions" (methods, procedures, etc) in non-functional languages which can perform side effects like reading/writing a file, accessing the network.

Purity provides a number of significant benefits, the biggest (in my opinion) being that, especially when coupled with immutability, reasoning about your code becomes drastically easier.  If you are debugging a function (perhaps not written by you) in a language with pervasive side effects then you might end up having to trace through every function call that it makes to find where some problem is occuring.  But in a pure function you can see that the only result of the function is the value returned.  

We'll cover this in more detail in the section below on Explicit Effects but the canonical example usually involves some code that has a side-effect of firing missiles somewhere.  If you're trying to track down the place where the missiles are inexplicibly getting fired, you can instantly ignore every pure function doesn't have `IO` in it's signature.

Another advantage of pure function is that since the same inputs always result in the same outputs they are trivially memoizable (and the results of trivially cachable in other situations).

I ran out of time to cover equational reasoning but this is [a great post by Gabriel Gonzalez](http://www.haskellforall.com/2013/12/equational-reasoning.html) that covers the topic pretty well.

On the last benefit listed above, without purity parametricity (the next topic) wouldn't be possible, because reflection and similar mechanism are side-effects that can be used to inspect parameteric values and discover their concrete types and then use their individual values, which would break parametricity.

## Parametric Polymorphism / Parametricity

- BENEFIT: Reduces the space of things to think about (eg space errors can hide in)
- BENEFIT: Allows type signature to serve as precise executable compiler verified documentation.

Parametric polymorphism refers to functions that have type variables which are not constrainted by any typeclass constraints.

The simplest example of such a function, and one of the canonical examples for discussions of parametric polymorphism is the identity function:

In [13]:
:t identity     -- NB. This is from `Protolude, in the default Prelude this function is called simply `id`

identity :: forall a. a -> a

The type of the identity function tells us that given any value of a given type, the function will give you back a value of the same type.

How many possible implementations of this function are there?  Well, there are actually several, but all of them except one are "bottoms" meaning they crash or fail in some way -- [Fast and Loose Reasoning is Morally Correct](https://www.cs.ox.ac.uk/jeremy.gibbons/publications/fast+loose.pdf) tells us that in general we can ignore these types of values in this type of discussion, but for completeness sack I'll present a few of the obvious bad examples:

In [14]:
:ext ScopedTypeVariables
:ext OverloadedStrings

In [15]:
badIdentityError :: forall a. a -> a
badIdentityError = panic "bad identity"

badIdentityUndefined :: forall a. a -> a
badIdentityUndefined = undefined

badIdentityHang :: forall a. a -> a
badIdentityHang = badIdentityHang

I think this is just about it for the range of possible bad implementations.  That leaves us with one possible good implementation:

In [16]:
goodIdentity :: forall a. a -> a
goodIdentity x = x

Which, of course, is what we expect from an identity function and is how the identity function is actually defined in Haskell.

The interesting part here is that we can infer just from the type signature the exact implementation of the function.  Of course it's possible to write parametricly polymorphic functions for which you cannot deduce the entire implementation from the type signature but regardless of how complicated they get each unconstrained type variable eliminates a huge chunk of the space of possible implementations allowing for easier reasoning.

As another example, compare the type signatures `[Int] -> Int` vs `[a] -> Int`.  There are numerous obvious non-bottom implementations of `[Int] -> Int`, eg:

- `head`
- `last`
- `length`
- `maximum`
- `minimum`

However if we make our signature more general so that it can handle any type of list, rather than just a list of `Int`s there is only one reasonable non-bottom implementation of `[a] -> Int`, namely... ?

## Totality

- BENEFIT: Makes type signatures more honest (better from documentation point of view)
- BENEFIT: Less error handling code (try/catch) sprinkled throughout business logic
- BENEFIT: Enables confident composition of small pieces 

Haskell isn't actually total (like Idris or Agda) but in practice you can exist in a largely total world and it's relatively easy to get a grip on what is and is not total and constrain yourself largely to the total space whenever possible.  

Though there are a few places where it inevitably creeps back in, the obvious one being division by zero.  The reason for this is that it's simply too convenient to be able to write expressions in mathematical notation, but if you are working in a domain where the issue came up frequently it's easy enough to implement your own safe division operator and use that:

In [17]:
safeDiv :: (Eq a, Fractional a) => a -> a -> Maybe a
safeDiv n 0 = Nothing
safeDiv n d = Just $ n / d

print (safeDiv 5 2)
print (safeDiv 3 0)

Just 2.5

Nothing

Totality is liberating.  Probably the simplest motivating example of this is that total functions cannot throw exceptions or crash in any other way.  How happy would you be if I told you that none of your python or Java functions can throw exceptions or crash anymore?

See again [Fast and loose reasoning is morally correct](https://www.cs.ox.ac.uk/jeremy.gibbons/publications/fast+loose.pdf) for more details.

## Explicit Effects

- BENEFIT: Enables composition (and first class manipulation) of effectful computations
- BENEFIT: Allows pushing as much as possible into pure functions to simplify considerations when coding

This is related closely to purity.  I don't have the time I wanted to go into this but the general idea is that functions like `print` which cause a side-effect (output to the console in this case) must indicate this fact in their type signature:

In [18]:
:t print

print :: forall (m :: * -> *) a. (Show a, MonadIO m) => a -> m ()

This type signature is actually a little more complicated than it would normally be because we're using the version of `print` from `Protolude` which is more versatile than the built in one.  Let's check the type of the built in `print`:

In [19]:
:t P.print

P.print :: forall a. Show a => a -> IO ()

Again dropping the forall which IHaskell adds, we get simply `Show a => a -> IO ()`.  This signature says "given a value of type `a` which must be Showable, this function will execute an IO action which returns `()` (unit).  Whenever you see a signature of `IO ()` you immediately know that a side-effect is happening.

An important thing to note here is that the IO action gets executed by the Haskell runtime at... well, runtime.  Until then, IO actions are still first class values that can be passed around to other functions and combined into new actions.  For example:

In [20]:
threeWaysToPrintAString = [print, putStrLn, putStr]

:t threeWaysToPrintAString

threeWaysToPrintAString :: forall (m :: * -> *) a. (MonadIO m, Show a, Print a) => [a -> m ()]

This type signature says essentially that it's a list of functions that take a Showable a and return unit in a some `MonadIO` context, which is just a generalized version of IO (due to `Protolude` again in this case).

Now that we have this list of functions we can apply them all to some value:

In [21]:
mapM ($ "foo") threeWaysToPrintAString

"foo"
foo
foo[(),(),()]

We get the `print`'d version which calls Show on the string adding the quotes, the `putStrLn`'d version which just prints the string followed by a newline, then the `putStr`'d version which prints the string without the newline and then we see the return value of the `mapM` call which collected the three unit values returned by each of the IO actions.

Using [MTL style](https://ocharles.org.uk/posts/2016-01-26-transformers-free-monads-mtl-laws.html) you can write more descriptive type signatures that differentiate at a more granular level than just IO vs non-IO.

You could imagine a function with a type signature something like this:

    fetchUserFromDb :: ( MonadReader DbConn m, MonadReadDb m, MonadLogger m )
    
This would tell you that the function must be called in a context that provides some sort of configuration containing a `DbConn` value, and that it can query (but not write to) the database, and that it can log messages.  But it can't for example, make a network connection, read or write files on disk, talk to redis or cassandra, etc.

One interesting thing to note is that functions which return IO actions and other side effecting types are still pure in Haskell.  Given the same input they will always produce the same IO action -- it's only at runtime when the IO action is executed by the Haskell runtime that different paths will be taken based on the state of the outside world at the time the code is executing.

## Laziness

- BENEFIT: Allows many algorithms to be expressed more naturally
- BENEFIT: Simplifies a lot of code (eg throw everything in where clauses and only those that are forced are evaluated)

Ran out of time to cover this in depth but an example of how it's useful in Haskell that goes a little beyond our `fibs` example from the introduction is the concept of "tying the knot" which lets you build mutually recursive data structures which would otherwise not be possible due to immutability.

For example here is a quick doubly linked list:

In [22]:
data DubLL a
  = Nil
  | Node a (DubLL a) (DubLL a)
  deriving (Eq, Ord, Read, Show)

toList :: DubLL a -> [a]
toList Nil = []
toList (Node x _ next) = x:toList next

-- A finite list with no cycles
dubLL :: DubLL Int
dubLL = a
  where
    a = Node 1 Nil b
    b = Node 2 a c
    c = Node 3 b d
    d = Node 4 c Nil

-- The same list connected head to tail to form an infinite cycle in both directions
cycLL :: DubLL Int
cycLL = a
  where
    a = Node 1 d b
    b = Node 2 a c
    c = Node 3 b d
    d = Node 4 c a
    
print (take 20 $ toList dubLL)
print (take 20 $ toList cycLL)    

[1,2,3,4]

[1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4]

## Immutability

BENEFITS: Local reasoning
BENEFITS: A whole (significant) class of errors can no longer be made

Immutability is probably the most important thing that I ran out of time to write about in depth, but it's starting to get so popular these days that maybe you've encountered it elsewhere.  If not, hopefully it's obvious how having data that can't change makes it easier to reason about the behavior of your code -- you can pass values you are holding on to to any other function and not worry that they will be changed out from under you.

If you haven't encountered it before you might have the obvious question on your mind:

### How can you do anything if you can change any values?

The short answer is that you create a new copy with the change, so for example:

In [23]:
xs = [1..10]
ys = filter even xs

print xs
print ys
print xs

[1,2,3,4,5,6,7,8,9,10]

[2,4,6,8,10]

[1,2,3,4,5,6,7,8,9,10]

As you can see, the filter operation didn't change `xs`, it just created a new list, `ys`.  

Your next question might be:

### Isn't making all thoe copies expensive? 

The short answer is: no.  The slightly longer answer is: No, due to structural sharing in [persistent data structures](https://en.wikipedia.org/wiki/Persistent_data_structure).

## Pattern Matching

- BENEFIT: Expressiveness
- BENEFIT: Allows compiler to alert on missing cases

Pattern matching doesn't often get elevated to the level of some of these other features in discussions of the benefits of Haskell but I think it deserves more credit.  Maybe part of the reason is that by default Haskell doesn't force you to handle every case, but everyone that sticks with Haskell eventually enables at least the "-Wall" flag which will warn you on an incomplete pattern match if not "-Werror" which will force you to handle every case.

A specific example of when this benefit mainfests is when you add a new constructor to a sum type -- the compiler will instantly tell you where all of the places that you need to update to handle to the new constructor.

# Examples

## The StreamSource Algebraic Data Type

This is a data type I created for a library I was working on to do event sourcing in Haskell.  You can see it [in context here](https://github.com/SuperpowersCorp/dispenser-core/blob/master/src/Dispenser/Types.hs#L105).

The basic idea is that you need to be able to select which set of streams you want to read from and you might want to combine several different reads into a single pass over the data.  In order to do this you need to decide the canonical set of streams that you need to read from to get all of the data you need for any of the individual reads.

As long as each individual read requires data from a specific set of streams then a `Set` of stream names is a good way to represent what you need, but as soon as one of the reads requires reading from all streams then it no longer matters what specific streams any of the other reads requires, you're going to have to fetch all of the data anyway.

In another language I might be tempted to do something like use a set of stream names, and as you're building up the full set to read from if you ever get an empty set, treat this as a special sigil value that indicates that the read needs events from all streams in which case discard the current set and just hold the empty set as that sigil value, then as you add each additional stream source, check and see if the current set is empty in which case just discard the new ones, and if it's not then union them in.. the logic might look something like this pseudo code:

    sources = ...
    finalSource = sources.first()
    
    // empty set indicates "all streams" so if we encounter an empty set at any point everything becomes
    // an empty set
    for source in sources.rest():
      if (finalSource.empty() || source.empty()) {
        finalSource = emptySet()
      } else {
        finalSource = finalSource.union(source)
      }
      
    if (finalSource.empty()) {
      return readAllEvents()
    } else {
      return readEventsFrom(finalSource.toList())
    }
    
This really isn't that hard to write, test and debug... but presumably it's implemented once somewhere in a function exposed from some API somewhere called something like "readEventsFromSources"...

The algebraic approach in Haskell was:

In [24]:
:ext DeriveDataTypeable
:ext DeriveGeneric

import           Data.Data              ( Data )
import qualified Data.Set       as Set
import qualified Data.Semigroup as Semi
import           Data.Zero      

newtype StreamName = StreamName { unStreamName :: Text }
  deriving (Data, Eq, Generic, Ord, Read, Show)

data StreamSource
  = AllStreams
  | SomeStreams (Set StreamName)
  deriving (Data, Eq, Generic, Ord, Read, Show)

instance Semigroup StreamSource where
  SomeStreams a <> SomeStreams b = SomeStreams (a <> b)
  AllStreams    <> _             = AllStreams
  _             <> AllStreams    = AllStreams

instance Monoid StreamSource where
  mempty = SomeStreams mempty
  mappend = (Semi.<>)
  
instance Zero StreamSource where
  zero = AllStreams

Everything used to build this comes with Haskell except for the `Zero` class and it's `zero` function which are from the [zero](http://hackage.haskell.org/package/zero-0.1.4/docs/Data-Zero.html) package which implements "Semigroups with absorbing element".  Haskell already has Semigroups as we covered in the second part (which, as a reminder, just means any type that has a binary operation that allows you to combine any two elements of the type to get a new element of the type) so really all the `zero` package adds is the notion of an absorbing element, which is exactly what we want for our `AllStreams` case.

This allows us to smush any to `StreamSource`s together at any point in our program (not just when we call a specific `readEventsFromSources` function) and wind up with a new single valid `StreamSource` that describes what we want:

In [25]:
:ext OverloadedStrings

abcStreams = SomeStreams (Set.fromList . map StreamName $ ["a", "b", "c"])
defStreams = SomeStreams (Set.fromList . map StreamName $ ["d", "e", "f"])

print abcStreams
print defStreams
print (abcStreams <> defStreams)
print (abcStreams <> defStreams <> abcStreams <> abcStreams <> defStreams <> defStreams)
print AllStreams
print (AllStreams <> abcStreams)
print (defStreams <> AllStreams)
print (abcStreams <> AllStreams <> defStreams)

SomeStreams (fromList [StreamName {unStreamName = "a"},StreamName {unStreamName = "b"},StreamName {unStreamName = "c"}])

SomeStreams (fromList [StreamName {unStreamName = "d"},StreamName {unStreamName = "e"},StreamName {unStreamName = "f"}])

SomeStreams (fromList [StreamName {unStreamName = "a"},StreamName {unStreamName = "b"},StreamName {unStreamName = "c"},StreamName {unStreamName = "d"},StreamName {unStreamName = "e"},StreamName {unStreamName = "f"}])

SomeStreams (fromList [StreamName {unStreamName = "a"},StreamName {unStreamName = "b"},StreamName {unStreamName = "c"},StreamName {unStreamName = "d"},StreamName {unStreamName = "e"},StreamName {unStreamName = "f"}])

AllStreams

AllStreams

AllStreams

AllStreams

We utilize [QuickCheck tests here](https://github.com/SuperpowersCorp/dispenser-core/blob/master/test/TypesSpec.hs#L13) to validate that the laws for Monoid (which encompass the laws for Semigroup) as well as the laws for Zero hold for our type.  (We could take this even further in Idris and write a proof that they always hold rather than just testing on 100 or 1000 cases).

We end up (via a very small amount of code) with a type which is guaranteed to be correct by construction -- you can stil make logical errors like trying to read from the wrong stream and of course you might make a typo in a stream name but you no longer have to worry about users trying to implement all the logic to make sure they handle empty sets properly any time they combine stream sources outside of your special `readEventsFromSources` function.

## Fun with Optics (Lenses)

A thorough review of everything interesting about lenses would be way to long for this already way too long notebook, so I'll sum them up by saying that they are basically getter/setters on steroids.

The getter/setter part is most obvious when applied to record types like we have seen before:

In [26]:
import Control.Lens ( (^.)
                    , (^..)
                    , (.~)
                    , (%~)
                    , (+~)
                    , (*~)
                    , (^?)
                    , _1
                    , _2
                    , _3
                    , _4
                    , _Just
                    , _Nothing
                    , _head
                    , Iso'
                    , Lens'
                    , Prism'
                    , Traversal' 
                    , allOf
                    , each
                    , filtered
                    , indices
                    , iso
                    , itraversed
                    , ix
                    , lens
                    , over
                    , partsOf
                    , prism
                    , re
                    , set
                    , traversed
                    , view
                    )
import Data.Data.Lens ( biplate )                    

In [27]:
data Profile = Profile
  { _username    :: Text
  , _friendCount :: Int
  , _enemyCount  :: Int
  , _address     :: Address
  } deriving (Data, Eq, Generic, Ord, Read, Show)
  
data Address = Address
  { _street  :: Text
  , _poBox   :: Maybe Int  -- You shouldn't use Int's for P.O. Boxes or Zip Codes, but makes for a better demo
  , _city    :: Text
  , _state   :: Text
  , _zipCode :: Int       
  } deriving (Data, Eq, Generic, Ord, Read, Show)
  
-- Normally we would use TemplateHaskell to generate all of these for us, but TemplateHaskell doesn't work
-- in notebooks yet, so for this example we will write them out by hand.  The TemplateHaskell version would
-- look like this:
--
-- makeLenses ''Profile
-- makeLenses ''Address
-- 
-- and would generate all of these for us:
  
username :: Lens' Profile Text 
username = lens _username (\profile un -> profile { _username = un })

friendCount :: Lens' Profile Int
friendCount = lens _friendCount (\profile fc -> profile { _friendCount = fc })

enemyCount :: Lens' Profile Int
enemyCount = lens _enemyCount (\profile ec -> profile { _enemyCount = ec })

address :: Lens' Profile Address
address = lens _address (\profile ad -> profile { _address = ad})

street :: Lens' Address Text
street = lens _street (\address st -> address { _street = st })

poBox :: Lens' Address (Maybe Int)
poBox = lens _poBox (\address po -> address { _poBox = po })

city :: Lens' Address Text
city = lens _city (\address ci -> address { _city = ci })

state :: Lens' Address Text
state = lens _state (\address st -> address { _state = st })

-- I'm going to leave the lense for zipCode out to demonstrate that biplate will still target it below.

With lenses in hand we can use them as simple traditional getter/setters:

In [28]:
prof = Profile "john" 5 0 (Address "4040 N. Fairfax Drive" (Just 123) "Arlington" "VA" 22203)
prof

Profile {_username = "john", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

In [29]:
prof ^. username

john

In [30]:
prof ^. friendCount

5

In [31]:
prof ^. enemyCount

0

In [32]:
let ad = prof ^. address

ad

Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}

In [33]:
ad ^. street

4040 N. Fairfax Drive

In [34]:
ad ^. poBox

Just 123

NB. Since poBox is of type `Maybe Int` we get back a `Maybe Int` here, hence the `Just` constructor.  We will see in the section below on setters how we can use prisms to deal with just the values that are present (or not present).

In [35]:
ad ^. city

Arlington

And so on... likewise for setters:

In [36]:
prof & username .~ "jake"

Profile {_username = "jake", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

In [37]:
prof & friendCount .~ 100

Profile {_username = "john", _friendCount = 100, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

In [38]:
ad & street .~ "123 Main Street"

Address {_street = "123 Main Street", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}

As we mentioned above, when dealing with optional (`Maybe`) values (or any other vlaue that may or may not be there, such as `Either` values which are often used to hold either a valid value of one type or an error value of another type):

In [39]:
-- Let's remind ourselves what our address looks like:
print ad

-- And create a copy with no poBox:
noPoAd = ad & poBox .~ Nothing

print noPoAd

Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}

Address {_street = "4040 N. Fairfax Drive", _poBox = Nothing, _city = "Arlington", _state = "VA", _zipCode = 22203}

Now we can use the `_Just` prism to address just the cases where the value is present:

In [40]:
print $ ad     & poBox . _Just .~ 123
print $ noPoAd & poBox . _Just .~ 12345

Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}

Address {_street = "4040 N. Fairfax Drive", _poBox = Nothing, _city = "Arlington", _state = "VA", _zipCode = 22203}

There is a corresponding `_Nothing` prism but sadly it doesn't quite do what you might hope and let you address the case where the value is not present.  There are various alternatives for this some of which are more or less helpful in various situations, for example `filtered` is helpful in this situation:

In [41]:
print $ ad     & poBox . filtered null .~ Just 5
print $ noPoAd & poBox . filtered null .~ Just 5
print $ ad     & poBox . filtered (not . null) .~ Just 5
print $ noPoAd & poBox . filtered (not . null) .~ Just 5

Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}

Address {_street = "4040 N. Fairfax Drive", _poBox = Just 5, _city = "Arlington", _state = "VA", _zipCode = 22203}

Address {_street = "4040 N. Fairfax Drive", _poBox = Just 5, _city = "Arlington", _state = "VA", _zipCode = 22203}

Address {_street = "4040 N. Fairfax Drive", _poBox = Nothing, _city = "Arlington", _state = "VA", _zipCode = 22203}

In [42]:
-- We can set also several fields at once...
ad & city    .~ "Washington"
   & state   .~ "DC"

Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Washington", _state = "DC", _zipCode = 22203}

But we can also use other lens operators to perform specific types of modifications, for example instead of directly setting a field we can apply functions to modify them:

In [43]:
import qualified Data.Text as T

print $ prof & username %~ T.reverse
print $ prof & username %~ T.toUpper

Profile {_username = "nhoj", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

Profile {_username = "JOHN", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

Or use mathematical operations to manipulate numbers directly:

In [44]:
print $ prof & friendCount +~ 10
print $ prof & friendCount *~ 10

Profile {_username = "john", _friendCount = 15, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

Profile {_username = "john", _friendCount = 50, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

And at the end of the day lenses are just functions and functions compose, so lenses compose:

In [45]:
print $ prof & address . city   %~ T.toUpper
print $ prof & address . street %~ T.reverse

Profile {_username = "john", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "ARLINGTON", _state = "VA", _zipCode = 22203}}

Profile {_username = "john", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "evirD xafriaF .N 0404", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

There are also regular function versions of each of the operators, which are often used with higher order functions or other places where they read more naturally than operator soup:

In [46]:
print $ prof ^. username
print $ view username prof

"john"

"john"

In [47]:
print $ prof & username .~ "Jake"
print $ set username "Jake" prof

Profile {_username = "Jake", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

Profile {_username = "Jake", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

In [48]:
print $ prof & username %~ T.toUpper
print $ over username T.toUpper prof

Profile {_username = "JOHN", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

Profile {_username = "JOHN", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

I don't think there is an operatorless way to replace the mathy operators like `+~` and `*~` but you can use `over` with sections to get by:

In [49]:
print $ prof & friendCount +~ 12
print $ over friendCount (+12) prof

Profile {_username = "john", _friendCount = 17, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

Profile {_username = "john", _friendCount = 17, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

More interestingly we can have lenses that use the `Data` and `Generic` classes that we derived when we created our data types to operate in a generic way on all values of a certain type:

In [50]:
prof & biplate %~ (T.toUpper . T.reverse)

Profile {_username = "NHOJ", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "EVIRD XAFRIAF .N 0404", _poBox = Just 123, _city = "NOTGNILRA", _state = "AV", _zipCode = 22203}}

In [51]:
prof & (biplate :: Data a => Traversal' a Int) *~ 33 

-- Not sure why I had to give the type signature here... even if this were always necessary,
-- for bigger more complicated examples this type signature is a relatively minor cost

Profile {_username = "john", _friendCount = 165, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 4059, _city = "Arlington", _state = "VA", _zipCode = 732699}}

As the type signature above says, the particular type of optic being employed by `biplate` is a `Traversal` and traversals also allow us to get multiple values as well (by adding an extra `.` to our `^.` operator) and specifying the type (this isn't always necessary if we are composing `biplate` with other lenses that determine the types):

In [52]:
prof ^.. (biplate :: Data a => Traversal' a Int)

[5,0,123,22203]

This is all of the ints from our structure: `friendCount`, `enemyCount`, `poBox` and `zipCode`.

Here is a (rather trivial) example where it's not necessary to specify the types:

In [53]:
prof ^.. biplate . street

4040 N. Fairfax Drive

While this example is trivial it will work across any deeply nested structure.

For example:

In [54]:
("foo", 1, [("bar", [(2, ('a', 3))]), ("baz", [(4,('b', 5))])]) & biplate *~ 100

("foo",100,[("bar",[(200,('a',300))]),("baz",[(400,('b',500))])])

NB. The `biplate` examples do not depend on the existence of the lenses we created, it would target the same parts of the structure even if we had not created any lenses manually.  It's purely based on what it can tell about the structure from `Data` class implementations.


So we've seen some ways in which lenses are at least a bit more powerful than regular getter/setters but as long as you make sure your various optics follow the lens laws, which are these:

- Get-Put: If you modify something by changing its subpart to exactly what it was before... then nothing happens
- Put-Get: If you modify something by inserting a particular subpart and then viewing the result... you'll get back exactly that subpart
- Put-Put: If you modify something by inserting a particular subpart a, and then modify it again inserting a different subpart b... it's exactly as if you only did the second step.

then you will have what are called "very well behaved" lenses and they can do all sorts of interesting things.

Here's a quick runthrough of some of the possibilities:

In [55]:
import Numeric.Lens (base, hex, octal)

In [56]:
"100" ^? base 16

Just 256

In [57]:
"100" ^? hex

Just 256

In [58]:
"100" ^? octal

Just 64

In [59]:
"100" ^? base 2

Just 4

In [60]:
"100" ^? base 10

Just 100

In [61]:
1767707668033969 ^. re (base 36)

"helloworld"

In [62]:
absVal :: Real a => Lens' a a
absVal = lens get set
  where
    get     = abs
    set n x = signum n * x

In [63]:
[    5   ^. absVal
 , (-5)  ^. absVal
 ,   3   ^. absVal
 , (-3)  ^. absVal
 , (-8)   & absVal .~ 4
 ,   5    & absVal %~ (*2)
 , (-5)   & absVal %~ (*2)
 ]

[5,5,3,3,-4,10,-10]

In [64]:
import qualified Data.Digits as DD

toDigits   = DD.digits   10
fromDigits = DD.unDigits 10
                 
digits :: Iso' Integer [Integer]
digits = iso toDigits fromDigits

In [65]:
toDigits 123456

[1,2,3,4,5,6]

In [66]:
fromDigits [1,2,3]

123

In [67]:
fromDigits [10, 15, 20, 25]

11725

In [68]:
8675309 ^. digits

[8,6,7,5,3,0,9]

In [69]:
8675309 & digits . each .~ 9

9999999

In [70]:
8675309 & digits . each +~ 1

9786420

In [71]:
8675309 & digits .~ [1,2,3]

123

In [72]:
8675309 & digits %~ reverse

9035768

In [73]:
-- The Control.Lens.Prism documentation provides a prism for 
-- accessing only natural numbers:

import Numeric.Natural ( Natural )

nat :: Prism' Integer Natural
nat = prism toInteger $ \ i ->
   if i < 0
   then Left i
   else Right (fromInteger i)

In [74]:
[1,-1,2,-2,3,-3,4,-4] & each . nat %~ (*2)

[2,-1,4,-2,6,-3,8,-4]

In [75]:
-- I guess it's more succinct than:
[1,-1,2,-2,3,-3,4,-4] & each . filtered (>0) %~ (*2)

[2,-1,4,-2,6,-3,8,-4]

In [76]:
-- ...though:

myNat = filtered (>0)

[1,-1,2,-2,3,-3,4,-4] & each . myNat %~ (*2)

[2,-1,4,-2,6,-3,8,-4]

In [77]:
[1..10] & ix 3 +~ 7

[1,2,3,11,5,6,7,8,9,10]

<hr>

Everything below this line through the end of the section is just pulled fairly directly from [John Wiegly's excellent talk Putting Lenses to Work](https://www.youtube.com/watch?v=QZy4Yml3LTY).

<hr>

In [78]:
867509 & digits . ix 4 +~ 7

867579

In [79]:
prof -- remind ourselves

Profile {_username = "john", _friendCount = 5, _enemyCount = 0, _address = Address {_street = "4040 N. Fairfax Drive", _poBox = Just 123, _city = "Arlington", _state = "VA", _zipCode = 22203}}

In [80]:
data Person = Person
  { _name :: Text
  , _age  :: Int
  } deriving (Data, Eq, Generic, Ord, Read, Show)

age :: Lens' Person Int
age = lens _age (\person a -> person { _age = a })

people = [Person "Alice" 33, Person "Bob" 52, Person "James" 29, Person "Susan" 41]

print $ allOf (each.age) even people
print $ allOf (each.age) (>21) people

False

True

In [81]:
(1,2,3) & _1 .~ 10
        & _2 .~ 20
        & _3 .~ 30

(10,20,30)

These `_N` style prisms work on many types of things that have a variable number of slots but they are convenient for accessing tuples in a size independent way (as opposed to the builtin `fst` and `snd` which are specific to 2-tuples):

In [82]:
:t fst
:t snd

fst :: forall b a. (a, b) -> a

snd :: forall a b. (a, b) -> b

Remember when we said not to treat tuples as lists... well, if you really want to:

In [83]:
(1,2,3) ^.. each

[1,2,3]

In [84]:
(1,2,3) & each *~ 2

(2,4,6)

Of course this doesn't work if the types of all the slots are not of the same type:

In [85]:
(1, 'b', "foo") & each *~ 2

<interactive>:1:10: error:
    • No instance for (IsString Char) arising from the literal ‘"foo"’
    • In the expression: "foo"
      In the first argument of ‘(&)’, namely ‘(1, 'b', "foo")’
      In the expression: (1, 'b', "foo") & each *~ 2
<interactive>:1:19: error:
    • No instance for (Num Char) arising from a use of ‘*~’
    • In the second argument of ‘(&)’, namely ‘each *~ 2’
      In the expression: (1, 'b', "foo") & each *~ 2
      In an equation for ‘it’: it = (1, 'b', "foo") & each *~ 2

In [86]:
[2,4,1,5,3,6] & partsOf (traversed.indices odd) %~ reverse

[2,6,1,5,3,4]

In [87]:
[2,4,1,5,3,6] & partsOf (each.filtered (<4)) %~ reverse.sort

[3,4,2,5,1,6]

In [88]:
-- Less impressive if you're not already a Haskell programmer... doing this without lenses would be laborious.

(3,1,2,4,5) & partsOf each %~ sort

(1,2,3,4,5)

In [89]:
import Data.String ( String )
import qualified Data.Char as C

str :: String -> String
str = identity

str "Hello World" & partsOf (each.filtered C.isAlpha) .~ "Howdy!!!"

"Howdy !!!ld"

In [90]:
((str "foo", str "bar"), "!", 2 :: Int, ()) ^.. biplate :: [String]

["foo","bar","!"]

In [91]:
:t ((str "foo", str "bar"), str "!", 2 :: Int, ())

((str "foo", str "bar"), str "!", 2 :: Int, ()) :: ((String, String), String, Int, ())

In [92]:
((str "foo", str "bar"), "!", 2 :: Int, ()) & biplate %~ C.toUpper

(("FOO","BAR"),"!",2,())

In [93]:
((str "foo", str "bar"), "!", 2 :: Int, ()) & partsOf biplate %~ (reverse :: String -> String)

(("!ra","boo"),"f",2,())

In [94]:
((str "foo", str "bar"), "baz!", 2 :: Int, ()) & (biplate :: Data s => Traversal' s String)._head %~ C.toUpper

(("Foo","Bar"),"Baz!",2,())

In [95]:
((str "foo", str "bar"), "!", 2 :: Int, ()) & partsOf (biplate . filtered (<= 'm')) %~ (reverse :: String -> String)

(("!oo","abr"),"f",2,())

## Duck Lens

I made some lenses to make using [Facebook's Duckling parser](https://github.com/facebook/duckling) easy:

In [96]:
import           Duck.Lens
import qualified Duck.Lens as DL  -- too disambiguate vs our `address` function from above

### Email Addresses

In [97]:
let emailText = "Please contact john@interos.net, bob@example.com and spy@whitehouse.gov about the secret meeting."

emailText ^.. emailAddresses . DL.address

john@interos.net

bob@example.com

spy@whitehouse.gov

In [98]:
emailText ^.. emailAddresses . user

john

bob

spy

In [99]:
emailText ^.. emailAddresses . domain

interos.net

example.com

whitehouse.gov

In [100]:
emailText & emailAddresses . DL.address .~ "[redacted]"

Please contact [redacted], [redacted] and [redacted] about the secret meeting.

In [101]:
emailText & emailAddresses . domain .~ "[redacted]"

Please contact john@[redacted], bob@[redacted] and spy@[redacted] about the secret meeting.

In [102]:
emailText & emailAddresses . user .~ "[redacted]"

Please contact [redacted]@interos.net, [redacted]@example.com and [redacted]@whitehouse.gov about the secret meeting.

In [103]:
emailText & emailAddresses . DL.address %~ T.toUpper

Please contact JOHN@INTEROS.NET, BOB@EXAMPLE.COM and SPY@WHITEHOUSE.GOV about the secret meeting.

In [104]:
emailText & emailAddresses . domain %~ T.toUpper

Please contact john@INTEROS.NET, bob@EXAMPLE.COM and spy@WHITEHOUSE.GOV about the secret meeting.

In [105]:
emailText & emailAddresses . user %~ T.toUpper

Please contact JOHN@interos.net, BOB@example.com and SPY@whitehouse.gov about the secret meeting.

### Phone Numbers

Caveats
- Prefixes and Extensions don't work
- Detection is overly broad (this is on Duckling).

In [106]:
let phoneStr = "303-555-1212 and (720) 444-2121 and also 9998887777... the end" 

phoneStr ^.. phoneNumbers . format

303-555-1212

(720) 444-2121

9998887777

In [107]:
phoneStr ^.. phoneNumbers . numbers

3035551212

7204442121

9998887777

In [108]:
phoneStr & phoneNumbers . numbers .~ "XXXXXXXXXX"

XXX-XXX-XXXX and (XXX) XXX-XXXX and also XXXXXXXXXX... the end

In [109]:
phoneStr & phoneNumbers . numbers .~ "8008675309"

800-867-5309 and (800) 867-5309 and also 8008675309... the end

In [110]:
phoneStr & phoneNumbers . format .~ "(111) 111-1111"

(303) 555-1212 and (720) 444-2121 and also (999) 888-7777... the end

In [111]:
phoneStr & phoneNumbers . numbers %~ T.reverse

212-155-5303 and (121) 244-4027 and also 7777888999... the end

### Numerals

In [112]:
numStr = "I've got ninety nine problems but a lens ain't one."

numStr ^.. numerals . asWritten

ninety nine

one

In [113]:
numStr ^.. numerals . asNumber

[99.0,1.0]

In [114]:
numStr & numerals . asWritten %~ T.reverse

I've got enin ytenin problems but a lens ain't eno.

In [115]:
numStr & numerals . asNumber +~ 10

I've got one hundred nine problems but a lens ain't eleven.

In [116]:
numStr & numerals . asNumber .~ 42.0

I've got forty-two problems but a lens ain't forty-two.

In [117]:
numStr & numerals . asWritten .~ "[redacted]"

I've got [redacted] problems but a lens ain't [redacted].

## Property-based Testing with QuickCheck

Property based testing exists at a point in between manual unit/UAT tests and proofs both on the spectrum of the value it provides and the effort that is required.

With manual testing the programmer thinks about ways in which the code under test might break and then implements individual tests to check each of the possible failure modes they can think of.  This usually involves thinking about boundary conditions -- if a number is accepted as input: what happens if it's negative? zero? very large? etc.  If it's a string what happens if it is empty? if it contains unicode? if it's very large? 

There are several problems with this approach.  The first is it only proves the specific cases that tests are written for don't occur -- it says nothing about the entire rest of the space of possibilities.  Secondly it is labor intensive as the more things you want to test the more tests you have to write.  Thirdly the tests are written by humans which can make mistakes in test code just as easily as they can in the code under test.  What's the solution to this? Do we need to write tests for our tests?

At the opposite end of the spectrum is formal verification of programs via proofs.  Proofs solve the first problem -- a valid proof proves that a program (or function, etc) works correctly for all possible inputs.  They are still very labor intensive, but they scale better -- the programmer doesn't have to create a bigger proof to get more certainty.  One the third point proofs done by hand would suffer the same problem as unit tests -- human fallibility -- but no one does formal verification by hand.  For a long time (at least as I understand it) most systems for formal verification such as [Coq](https://coq.inria.fr/) and [Isabelle](https://isabelle.in.tum.de/) were very difficult to work with and using proofs as a means of guaranteeing properties of practical programs was not considered feasible by many.

These days the situation has improved with languages like Idris and Agda that blur the line between language and proof systems, but in the meantime property testing strikes a good balance in that they can be used effectively today in Haskell (and other languages) but they provide a level of confidence that, while less than that provided by formal proofs, is much greater than that provided by hand written tests.

The way it works is that you essentially provide a predicate which describes the property you expect to hold and the property testing library (in our case [QuickCheck](https://hackage.haskell.org/package/QuickCheck)) generates a bunch of random inputs and tries them all, checking that the predicate holds.  QuickCheck uses some heuristics to try to do the same sort of things that a human developer would -- testing boundary conditions and so forth.

Let's see an example:

In [118]:
import           Test.QuickCheck
import qualified Test.QuickCheck as QC

never5_property :: (Eq a, Num a) => [a] -> Bool
never5_property = notElem 5

Now that we have a property we can check it with the `quickCheck` function:

In [119]:
quickCheck never5_property

*** Failed! Falsifiable (after 8 tests and 3 shrinks): 
[5]

We can see that (after 15 attempts) it found a counterexample -- the obvious one -- a list with just the number 5 in it.

The "4 shrinks" part refers to an interesting feature of QuickCheck in which, once QuickCheck finds a counter example it attempts to shrink it down to the smallest possible failing case.  We can see what's going on by using the `verboseCheck` function instead:

In [120]:
verboseCheck never5_property

Passed:
[]
Passed:
[-1]
Passed:
[]
Passed:
[3,0,0]
Passed:
[2,2]
Passed:
[-1,-3,-1,4]
Passed:
[-5,1]
Passed:
[]
Passed:
[0]
Passed:
[0,-1,-1,-5,-8,3,-8,6]
Passed:
[-8]
Passed:
[-10,8]
Passed:
[12,4,-5,1,3,-3,4,11,-10]
Passed:
[9,-2,-2,9]
Passed:
[-12,-6,-2,-2,3,-9,-6,0,-13,11,0]
Passed:
[0,12,-8,2,3,8]
Passed:
[8,12,-6,-12,-14,0,-8,-7,13,-8]
Passed:
[-2,10,4,-3,12,11,-14,-14,-4,3,8,-1]
Failed:
[6,11,-11,14,5]
*** Failed! Passed:
[]
Failed:
[-11,14,5]
Passed:
[]
Failed:
[14,5]
Passed:
[]
Failed:
[5]
Passed:
[]
Passed:
[0]
Passed:
[3]
Passed:
[4]
Falsifiable (after 19 tests and 3 shrinks): 
[5]

I'm not sure why it tries the empty list multiple times but we can see that it first found a list with multiple numbers in it, and then eventually managed to shrink it down to the minimal case.

As a side note, [hypothesis](https://github.com/HypothesisWorks/hypothesis) is a QuickCheck like testing framework for python that gets rave reviews and as I understand it has most of the features of QuickCheck (eg shrinking) -- though I have not used it myself.

## Haxl

[Haxl](https://github.com/facebook/Haxl) is a library developed at Facebook which describes itself as:

> Haxl is a Haskell library that simplifies access to remote data, such as databases or web-based services. Haxl can automatically
>
>    - batch multiple requests to the same data source,
>    - request data from multiple data sources concurrently,
>    - cache previous requests,
>    - memoize computations.
>
> Having all this handled for you behind the scenes means that your data-fetching code can be much cleaner and clearer than it would otherwise be if it had to worry about optimizing data-fetching. 
    
This is only really possible because Haskell allows us to encode effects in types such that it can know what is safe to cache/memoize/etc.  This could be done in other languages but only by building machinery that provides some of the guarantees that we get for free with Haskell's type system and then it would likely still be underpowered and probably consist of an awkward DSL that exists a level above the language itself, whereas with Haxl in Haskell you can implement normal "straight line code" that looks something like this:

```haskell
user           <- getUserById userId
friends        <- getFriends user
friendsFriends <- mapM getFriends friends
doSomethingWith friendsFriends
 where 
   getFriends user = mapM getUserById (friendsOf user)
```

and it will behave like normal evaluation by evaluating the first line first, since the user is a data dependency of the next getFriends call, but then the individual requests to fetch the friends which is written like a sequential map over the list of friends will automatically be run in parallel and likewise for the friend's friends... but furthermore if any of the friend's friends are also the user's friends then the requests for those friends will be cached and reused instead of being re-fetched.

## Dependently Typed Tensorflow

https://www.youtube.com/watch?v=ulwoUq6VaSs

https://mmhaskell.com/blog/2017/9/11/deep-learning-and-deep-types-tensor-flow-and-dependent-types


## Idris (Beyond Haskell)

Live demo illustrating:
- Dependent types
- Getting the compiler to write your code for you
- Proving properties about your code (take that, QuickCheck!)

## How to Get Started with Haskell

If I've managed to pique your curiosity enough that you want to try Haskell out, one of the first issues you will run into is whether to use `cabal (new-build)` or `stack`.  Cabal is the perennial build/package management tool for Haskell.  For a long time it was so bad that the experience of using it often resulted in what came to be known as "cabal hell" and you can still find many references to how to escape from cabal hell in Google.

To fix this, Michael Snoyman created [Stack](https://docs.haskellstack.org/en/stable/README/) which solved the vast majority of problems associated with cabal hell in one fell swoop.  Snoyman is a bit of controversial figure in the community and this (finally) led to great enough dissatisfaction on the behalf of the cabal developers that they created ["cabal new-build"](https://www.haskell.org/cabal/users-guide/nix-local-build-overview.html) to attempt to address the problems in their own way.

As I understand it these days it's roughly on parity with Stack's features though as a happy Stack user I haven't used it enough to be comfortable with it.  There's also a third option these days, which is using [the nix package manager](https://nixos.org/nix/).  The nix package manager was created for NixOS but can be used on any other unix operating system including OS X.  I am not sure about the level of support for windows. Nix, like haskell, has a fairly rough learning curve but I'm told that, like haskell, it is worth it -- but I haven't used it enough to testify to that point myself.

Beyond using Stack you'll have to deal with getting editor integration working.  There are at least three good options for Emacs: [Haskell Mode](https://github.com/haskell/haskell-mode), [Intero](https://github.com/commercialhaskell/intero) (no s) and [dante](https://github.com/jyp/dante).  I happen to use haskell-mode but that's largely just historical happenstance.

There seem to be plenty of Vim users seem to are happy with their Haskell setups but I have no knowledge of the Vim related tooling.

There are at least two good options for getting started quickly if you don't have a strong Editor/IDE preference -- the first is just to use whatever editor you want (probably with at least Haskell syntax highlighting installed) and then use [ghcid](https://github.com/ndmitchell/ghcid) as [described here](https://www.parsonsmatt.org/2018/05/19/ghcid_for_the_win.html).  Ghcid is lightweight, "just works" with any other tooling and should work on any platform.  The second is to use [VSCode with Haskelly](https://blogs.msdn.microsoft.com/uk_faculty_connection/2017/06/08/learning-haskell-and-using-vscode-and-haskelly/) which gives you a fairly typical featureful IDE setup.

And of course if you run into trouble or have any questions, please don't hesitate to reach out to me!